# Book Recommender System

- Content-based recommendation - uses supervised learning to induce a classifier to discriminate between interesting and uninteresting items for the user
- Collaborative filtering - recommends items based on similarity measures between users &/ items, asuuming users with similar interests have common preferences

In [2]:
# imports
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


## 1. Data Pre-processing

In [40]:
# TODO keep only the English contents "eng", en-US, en-GB, en-CA

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("books.csv")

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11127 non-null  int64  
 1   title               11127 non-null  object 
 2   authors             11127 non-null  object 
 3   average_rating      11127 non-null  float64
 4   isbn                11127 non-null  object 
 5   isbn13              11127 non-null  int64  
 6   language_code       11127 non-null  object 
 7     num_pages         11127 non-null  int64  
 8   ratings_count       11127 non-null  int64  
 9   text_reviews_count  11127 non-null  int64  
 10  publication_date    11127 non-null  object 
 11  publisher           11127 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


In [15]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In [42]:
df.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11127.000000,11127.000000,1.112700e+04,11127.000000,1.112700e+04,11127.000000
mean,21310.938887,3.933631,9.759888e+12,336.376921,1.793641e+04,541.854498
std,13093.358023,0.352445,4.428964e+11,241.127305,1.124794e+05,2576.176608
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10287.000000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780586e+12,299.000000,7.450000e+02,46.000000
75%,32104.500000,4.135000,9.780873e+12,416.000000,4.993500e+03,237.500000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [13]:
df.head(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,0976540606,9780976540601,en-US,152,19,1,4/26/2005,Nimble Books
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,28242,808,9/12/2005,Scholastic
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3628,254,11/1/2005,Gramercy Books
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,0345453743,9780345453747,eng,815,249558,4080,4/30/2002,Del Rey Books
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,1400052920,9781400052929,eng,215,4930,460,8/3/2004,Crown


In [14]:
df.tail(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
11117,45617,O Cavalo e o Seu Rapaz (As Crónicas de Nárnia ...,C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,3.92,9722330551,9789722330558,por,160,207,16,8/15/2003,Editorial Presença
11118,45623,O Sobrinho do Mágico (As Crónicas de Nárnia #1),C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,4.04,9722329987,9789722329989,por,147,396,37,4/8/2003,Editorial Presença
11119,45625,A Viagem do Caminheiro da Alvorada (As Crónica...,C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,4.09,9722331329,9789722331326,por,176,161,14,9/1/2004,Editorial Presença
11120,45626,O Príncipe Caspian (As Crónicas de Nárnia #4),C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,3.97,9722330977,9789722330978,por,160,215,11,10/11/2003,Editorial Presença
11121,45630,Whores for Gloria,William T. Vollmann,3.69,0140231579,9780140231571,en-US,160,932,111,2/1/1994,Penguin Books
11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11125,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco
11126,45641,Las aventuras de Tom Sawyer,Mark Twain,3.91,8497646983,9788497646987,spa,272,113,12,5/28/2006,Edimat Libros


In [22]:
df.loc[11]["title"]

"The Ultimate Hitchhiker's Guide (Hitchhiker's Guide to the Galaxy  #1-5)"

In [28]:
df.isnull().values.any()

False

In [32]:
print(df["language_code"].unique())
print(len(df["language_code"].unique()))

['eng' 'en-US' 'fre' 'spa' 'en-GB' 'mul' 'grc' 'enm' 'en-CA' 'ger' 'jpn'
 'ara' 'nl' 'zho' 'lat' 'por' 'srp' 'ita' 'rus' 'msa' 'glg' 'wel' 'swe'
 'nor' 'tur' 'gla' 'ale']
27


In [33]:
print(df["publisher"].unique())
print(len(df["publisher"].unique()))

['Scholastic Inc.' 'Scholastic' 'Nimble Books' ... 'Suma'
 'Panamericana Editorial' 'Editorial Presença']
2292


In [35]:
df_eng = df[df["language_code"] == "eng"]

In [38]:
df_eng

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11107,45572,Fantastic Mr. Fox (Cover to Cover),Roald Dahl,4.05,1855495090,9781855495098,eng,90,7,0,9/15/1995,BBC Audiobooks
11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books


In [37]:
len(df_eng["publisher"].unique())

1874

Content based filtering will be used, given that book information is known while user profiles are unknown

# 2. Feature Engineering

In [41]:
# TODO correlation or similarity metrics?

# 3. Recommendation Model

In [ ]:
# have only the english content
# do a mean, highest, lowest rating/length analysis df.describe()
# (find a dataset with gennres?)
# or do key words analysis in the title?

# 

In [ ]:
# reserve 10% for testing, 90% data for training the nearest neighbors model 

# calculate the similarity and disctance between any two books - with Nearest Neighbors autofit

# input a book, output it with 9 top most similar books

In [ ]:
# features: 
# keywords in title (word to vector)
# author
# publisher
# num-pages

In [41]:
textstr = df["title"][0]
print(type([textstr]))


<class 'list'>


In [40]:
from sklearn.feature_extraction.text import HashingVectorizer
# list of text documents
text = df["title"][0]
text = [text]
# create the transform
vectorizer = HashingVectorizer(n_features=20)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 20)
[[ 0.          0.60302269  0.          0.          0.         -0.30151134
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -0.60302269  0.
  -0.30151134 -0.30151134]]


In [66]:
titlelen = df["title"].str.len().tolist()
maxtr = max(titlelen)
maxid = titlelen.index(maxtr)
print(maxtr)
print(maxid)
df.iloc[[7455]]
df["title"][7455]

254
7455


"Yours in Food  John Baldessari: with meditations on eating by Paul Auster  David Byrne  Dave Eggers  David Gilbert  Tim Griffin  Andy Grundberg  John Haskell  Michael Kimmelman  Michael More  Glenn O'Brien  Francine Prose  Peter Schjeldahl  Lynne Tillman"

In [5]:
df["title"][2]

'Harry Potter and the Chamber of Secrets (Harry Potter  #2)'

In [15]:
from sklearn.feature_extraction.text import HashingVectorizer
feature = df["title"]
test_sample = df["title"][2]
vectorizer = HashingVectorizer(n_features=20)
feature_vec = vectorizer.transform(feature)
test_vec = vectorizer.transform([test_sample])

In [16]:
df[df["title"] == test_sample]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
4416,15881,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.42,0439064864,9780439064866,eng,341,2293963,34692,6/2/1999,Arthur A. Levine Books / Scholastic Inc.


In [47]:
feature

0        Harry Potter and the Half-Blood Prince (Harry ...
1        Harry Potter and the Order of the Phoenix (Har...
2        Harry Potter and the Chamber of Secrets (Harry...
3        Harry Potter and the Prisoner of Azkaban (Harr...
4        Harry Potter Boxed Set  Books 1-5 (Harry Potte...
                               ...                        
11122     Expelled from Eden: A William T. Vollmann Reader
11123                          You Bright and Risen Angels
11124                      The Ice-Shirt (Seven Dreams #1)
11125                                          Poor People
11126                          Las aventuras de Tom Sawyer
Name: title, Length: 11127, dtype: object

In [46]:
feature_vec

<11127x20 sparse matrix of type '<class 'numpy.float64'>'
	with 50385 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=21)
model_knn.fit(feature_vec)
distances, indices = model_knn.kneighbors(test_vec, 21, return_distance=True)
print(indices)

[[ 4416     2 10679     1     0   615  5298  1233     3  8446  2944  4903
   1089  6482  3989 10966  2498  9263  9330  6329  4033]]


In [14]:
df.iloc[indices[0]]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
4416,15881,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.42,0439064864,9780439064866,eng,341,2293963,34692,6/2/1999,Arthur A. Levine Books / Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
10679,43509,Harry Potter and the Goblet of Fire (Harry Pot...,J.K. Rowling,4.56,074754624X,9780747546245,eng,636,18754,906,7/8/2000,Bloomsbury
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
615,2005,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.57,0747584664,9780747584667,eng,768,1213,78,6/23/2006,Bloomsbury Publishing
5298,19129,Silos Politics and Turf Wars: A Leadership Fa...,Patrick Lencioni,4.05,0787976385,9780787976385,eng,224,2677,166,2/1/2006,Jossey-Bass
1233,4256,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.56,074757362X,9780747573623,eng,480,3141,140,7/1/2008,Bloomsbury UK
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
8446,32483,Shakespeare: The Biography,Peter Ackroyd,3.99,140007598X,9781400075980,eng,608,1541,131,11/14/2006,Anchor


In [14]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]
neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
neigh.fit(samples)
kneib= neigh.kneighbors([[0, 0, 1.3]], 2, return_distance=True)
print(kneib)
nbrs = neigh.radius_neighbors(
   [[0, 0, 1.3]], 0.4, return_distance=False
)
print(nbrs)
print(np.asarray(nbrs[0][0]))

(array([[0.3, 0.7]]), array([[2, 0]], dtype=int64))
[array([2], dtype=int64)]
2


In [24]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='brute').fit(X)
print(nbrs)
distances, indices = nbrs.kneighbors(X)#np.array([[-1,-1]])
print("indices:",indices) # ????
print("distances:",distances)

NearestNeighbors(algorithm='brute', n_neighbors=2)
indices: [[0 1]
 [1 0]
 [2 1]
 [3 4]
 [4 3]
 [5 4]]
distances: [[0.         1.        ]
 [0.         1.        ]
 [0.         1.41421356]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.41421356]]


In [6]:
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=21)
# fit
model_knn.fit(something)

NameError: name 'something' is not defined

In [ ]:
# return movie based on the user input

In [42]:
# TODO similarity metrics: cosine similarity??
# K-nearest neighbor!!
# https://www.geeksforgeeks.org/recommendation-system-in-python/
"""
Find similar movies using KNN
"""
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
	
	neighbour_ids = []
	
	movie_ind = movie_mapper[movie_id]
	movie_vec = X[movie_ind]
	k+=1
	kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
	kNN.fit(X)
	movie_vec = movie_vec.reshape(1,-1)
	neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
	for i in range(0,k):
		n = neighbour.item(i)
		neighbour_ids.append(movie_inv_mapper[n])
	neighbour_ids.pop(0)
	return neighbour_ids


movie_titles = dict(zip(movies['movieId'], movies['title']))

movie_id = 3

similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]

print(f"Since you watched {movie_title}")
for i in similar_ids:
	print(movie_titles[i])


# 4. Evaluation